In [14]:
import numpy as np 
import sys 
from io import open 
import os 
!{sys.executable} -m pip install nltk 
import nltk 
nltk.download('wordnet')
nltk.download('omw-1.4')
from logging import debug, info, warning, error
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lisakoopmans/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/lisakoopmans/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [15]:
class PARSE_LAYER: 
    SYM = 2 
    SEM = 3 
    CAT = 4 
    SNS = 5 
    ROL = 6 

In [16]:
def get_conll_blocks(in_file, split_lines=True, add_doc=False):
    '''Read a CoNLL formatted input file and return the list of lists per sentence/document'''
    docs = []
    cur_doc = []
    doc_ids = []
    num_lines = -1
    for line in open(in_file, 'r'):
        if not line.strip() and cur_doc:
            docs.append(cur_doc)
            cur_doc = []
            doc_ids.append(num_lines)
        elif line.strip().startswith('# newdoc'):
            # Keep track of start of new documents in doc_ids
            # We form a list of all sentences in docs, but at some
            # point we have to put multi-sent docs in a single file
            num_lines += 1
            if add_doc:
                cur_doc.append(line.strip())
        elif not line.strip().startswith('#') and line.strip():
            if len(line.split()) != 7:
                raise ValueError("Line should always consist of 7 layer-values, found {0}\n{1}".format(len(line.split()), line.strip()))
            if split_lines:
                cur_doc.append(line.split())
            else:
                cur_doc.append(line.strip())
    # Add left over one if there's not an ending last line
    if cur_doc:
        docs.append(cur_doc)
        doc_ids.append(num_lines)
    # If num_lines is never increased, this means that the # newdoc information was not added
    # In that case we just assume the default of 1 doc per block
    if num_lines == -1:
        info("Assuming 1 document per CoNLL block")
        doc_ids = range(0, len(docs))
    info("Extracted {0} sents, for {1} docs".format(len(docs), doc_ids[-1] + 1))
    return docs, doc_ids


In [17]:
def read_data(directory, filename):
     path = os.path.join(directory, filename)
     docs, docs_ids = get_conll_blocks(path)
     return docs

language = 'en'
standard = 'gold'

directory = os.path.join('./data/4.0.0', language, standard)
print(directory)

train_data = read_data(os.path.join(directory), 'train.conll')
test_data = read_data(os.path.join(directory), 'test.conll')

train_data_labels = [word[PARSE_LAYER.SNS] for sentence in train_data for word in sentence]
test_data_labels = [word[PARSE_LAYER.SNS] for sentence in train_data for word in sentence]

./data/4.0.0/en/gold


In [19]:
def baseline(train_data, test_data, train_data_labels, test_data_labels):
        train_data_lemmas = [word[PARSE_LAYER.SYM] for sentence in train_data for word in sentence]
        test_data_lemmas = [word[PARSE_LAYER.SYM] for sentence in train_data for word in sentence]
    
        pred = []
        for lemma in train_data_lemmas:
            syns = wn.synsets(lemma, lang='eng')
            if len(syns) > 0:
                pred.append(syns[0])
            else:
                pred.append('O')

            
baseline(train_data, test_data, train_data_labels, test_data_labels)

[] if
[] .
[] this
[] .
[] celestial~seasonings
[] .
[] raman~raghav
[] .
[] charles~de~gaulle
[] 1970
[] .
[] the
[] .
[] than
[] .
[] ,
[] .
[] stretch~out
[] for
[] the
[] .
[] come~upon
[] upon
[] of
[] the
[] .
[] and
[] .
[] that
